In [7]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Custom libraries
from Components.TrainModel import DataModule, TEMPUS
from Components.TickerData import TickerData
from Components.BackTesting import BackTesting

# Torch ML libraries
import torch
import torch.nn as nn
from torch.optim import AdamW

device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cuda":
    torch.backends.cuda.matmul.allow_tf32 = False
    torch.backends.cudnn.allow_tf32 = False

In [ ]:
#TODO: Feature importance with SHAP values and plot
#TODO: hyperparameter tuning
#TODO: buy signals become if prediction > current by some delta (~5%). Reverse is sell (decrease by some delta). Senstitvity analysis should be conducted to compare this delta level
#TODO: Use quantstats for a HTMl tearsheet
#TODO: market-regime detector with Hiden-markov model
#TODO: Add a Echo State Networks (ESN) layer to the model
#TODO: randomly sample 50 tickers, run backtest for all of them, and plot. take average sharpe ratio, and other metrics

In [ ]:
# Set the Wikipedia page title and section header
tickers = pd.read_html("https://en.wikipedia.org/wiki/Nasdaq-100")[4]
# Clean up the dataframe
tickers = tickers.iloc[:, [1]].to_numpy().flatten()

In [ ]:
#tickers = ['IONQ','QBTS','RGTI']
training_dfs = []
stocks_dfs = []
for ticker in tickers:
    training_data, raw_stock_data = TickerData(ticker,years=10,prediction_window=5).process_all()
    training_dfs.append(training_data)
    stocks_dfs.append(raw_stock_data)

training_data = pd.concat(training_dfs, ignore_index=False)
stock_data = pd.concat(stocks_dfs, ignore_index=False)

In [ ]:
#training_data.to_csv("Data/NASDAQ_100_TrainingData_v2.csv", index=True)
#stock_data.to_csv("Data/NASDAQ_100_StockData_v2.csv", index=True)

In [2]:
training_data = pd.read_csv("Data/NASDAQ_100_TrainingData_v2.csv")
training_data = training_data.set_index(training_data['Date']).drop(columns=['Date'])

In [3]:
stock_data = pd.read_csv("Data/NASDAQ_100_StockData_v2.csv")
stock_data = stock_data.set_index(stock_data['Date']).drop(columns=['Date'])

In [4]:
# Automatically get the number of features given my data_module object

#Best config: {'lr': 4.390449033248878e-05, 'hidden_size': 256, 'num_layers': 1, 'dropout': 0.3477694988633191, 'weight_decay': 0.0001801390872725824, 'batch_size': 16, 'window_size': 10, 'grad_clip_norm': 0.8393802881451728}

config = {
    "lr": 4.390449033248878e-05,
    "weight_decay": 0.0001801390872725824,
    "hidden_size": 256,
    "num_layers": 1,
    "dropout": 0.3477694988633191,
    "batch_size": 16,
    "window_size": 50,
    "clip_size": 0.8393802881451728,
    "epochs": 20,
    "device": "cuda" if torch.cuda.is_available() else "cpu"
}

data_module = DataModule(training_data, window_size=config["window_size"], batch_size=config["batch_size"])
config["input_size"] = data_module.num_features

# Instantiate the model
model = TEMPUS(config)
# Set up loss and optimizer
criterion = nn.MSELoss()
optimizer = AdamW(model.parameters(), lr=config["lr"], weight_decay=config["weight_decay"])
# Train Model
history = model.train_model(data_module.train_loader, data_module.test_loader, criterion, optimizer, config["epochs"])

Training Epochs:   0%|          | 0/20 [00:00<?, ?it/s]


Best MAPE: 4.04%


In [11]:
import os
import torch.onnx
import onnx

def export_model_to_onnx(model, save_name, data_loader, device, dynamic_axes=False):
    """
    Exports a model to ONNX format with support for static and dynamic axes.

    Parameters:
        model: The PyTorch model to export.
        save_name (str): File name for the exported ONNX file (excluding extension).
        data_loader: DataLoader containing sample input data.
        device (str): Device on which the model operates, e.g., 'cpu' or 'cuda'.
        dynamic_axes (bool): Whether to export with dynamic axes (default: False).
    """
    try:
        # Fetch a sample input tensor from the DataLoader
        example_inputs, _ = next(iter(data_loader))
        example_inputs = example_inputs.to(device)

        # Define ONNX file path
        save_path = os.path.join("Models", f"{save_name}.onnx")

        # Dynamic axes configuration (optional)
        dynamic_axes_dict = None
        if dynamic_axes:
            dynamic_axes_dict = {
                'input': {0: 'batch_size', 1: 'seq_len'},  # Dynamic batch and sequence length for input
                'output': {0: 'batch_size', 1: 'seq_len'}  # Dynamic batch and sequence length for output
            }

        # Export to ONNX format
        torch.onnx.export(
            model=model,
            args=example_inputs,  # Sample input tensor
            f=save_path,
            input_names=["input"],  # Name for the model input
            output_names=["output"],  # Name for the model output
            export_params=True,  # Include the model parameters in the ONNX file
            opset_version=16,  # ONNX version to export (ensure compatibility with your project)
            dynamic_axes=dynamic_axes_dict,  # Dynamic axis configuration
        )

        # Verify the ONNX model
        onnx_model = onnx.load(save_path)
        onnx.checker.check_model(onnx_model)
        print(f"Model successfully exported to {save_path}")

    except Exception as e:
        print(f"Error exporting model to ONNX: {str(e)}")

# Usage Example:
export_model_to_onnx(model, "TEMPUS_v3", data_module.test_loader, device, dynamic_axes=True)

C:\Users\Thomas Altmann\PycharmProjects\Project_DeepGreen\Components\TrainModel.py:79: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if seq_len % factor != 0:
C:\Users\Thomas Altmann\PycharmProjects\Project_DeepGreen\Components\TrainModel.py:128: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if features == lstm_features.size(2):  # If dimensions match
C:\Users\Thomas Altmann\anaconda3\Lib\site-packages\torch\onnx\symbolic_opset9.py:4244: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an

Model successfully exported to Models\TEMPUS_v3.onnx


In [25]:
import onnxruntime
import numpy as np

data_module = DataModule(training_data, window_size=50, batch_size=16)
example_inputs, _ = next(iter(data_module.test_loader))

onnx_inputs = [tensor.numpy(force=True) for tensor in example_inputs]

ort_session = onnxruntime.InferenceSession(
    "Models/TEMPUS_v3.onnx", providers=["CPUExecutionProvider"]
)

onnxruntime_input = {input_arg.name: input_value for input_arg, input_value in zip(ort_session.get_inputs(), onnx_inputs)}

# ONNX Runtime returns a list of outputs
onnxruntime_outputs = ort_session.run(None, onnxruntime_input)[0]


InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Invalid rank for input: input Got: 2 Expected: 3 Please fix either the inputs/outputs or the model.

In [ ]:
# Plot training metrics
training_fig = model.plot_training_history(training_data)
training_fig.show()

In [ ]:
# Get predictions
preds_df = model.get_predictions(training_data)
merged_df = pd.merge(stock_data, preds_df, on=['Date', 'Ticker'], how='inner')

In [ ]:
# Create a combined plot with stock prices and prediction markers
def plot_combined_predictions(data, ticker):
    # Filter for a particular ticker
    if type(ticker) == str:
        data = data[data['Ticker'] == ticker]
    else:
        return "Ticker provided is not a valid value"

    # Create figure
    fig = go.Figure()

    # Plot stock price trend line
    fig.add_trace(go.Scatter(
        x=data['Date'],
        y=data['Close'],
        mode='lines',
        name='Stock Price',
        line=dict(width=1)
    ))

    # Split signals by type and correctness
    buy_signals = data[data['Predicted'] == 2]
    sell_signals = data[data['Predicted'] == 1]
    hold_signals = data[data['Predicted'] == 0]

    # Correct/incorrect buy signals
    correct_buy = buy_signals[buy_signals['Predicted'] == buy_signals['Actual']]
    incorrect_buy = buy_signals[buy_signals['Predicted'] != buy_signals['Actual']]

    # Correct/incorrect sell signals
    correct_sell = sell_signals[sell_signals['Predicted'] == sell_signals['Actual']]
    incorrect_sell = sell_signals[sell_signals['Predicted'] != sell_signals['Actual']]

    # Correct/incorrect hold signals
    correct_hold = hold_signals[hold_signals['Predicted'] == hold_signals['Actual']]
    incorrect_hold = hold_signals[hold_signals['Predicted'] != hold_signals['Actual']]

    # Plot buy signals
    fig.add_trace(go.Scatter(
        x=correct_buy['Date'],
        y=data.loc[correct_buy.index]['Close'],
        mode='markers',
        name='Correct Buy Signal',
        marker=dict(symbol='triangle-up', size=10, color='green')
    ))

    fig.add_trace(go.Scatter(
        x=incorrect_buy['Date'],
        y=data.loc[incorrect_buy.index]['Close'],
        mode='markers',
        name='Incorrect Buy Signal',
        marker=dict(symbol='triangle-up', size=8, color='gray', opacity=0.2)
    ))

    # Plot sell signals
    fig.add_trace(go.Scatter(
        x=correct_sell['Date'],
        y=data.loc[correct_sell.index]['Close'],
        mode='markers',
        name='Correct Sell Signal',
        marker=dict(symbol='triangle-down', size=10, color='red')
    ))

    fig.add_trace(go.Scatter(
        x=incorrect_sell['Date'],
        y=data.loc[incorrect_sell.index]['Close'],
        mode='markers',
        name='Incorrect Sell Signal',
        marker=dict(symbol='triangle-down', size=8, color='gray', opacity=0.2)
    ))

    # Plot hold signals (using a different symbol)
    fig.add_trace(go.Scatter(
        x=correct_hold['Date'],
        y=data.loc[correct_hold.index]['Close'],
        mode='markers',
        name='Correct Hold Signal',
        marker=dict(symbol='circle', size=8, color='blue')
    ))

    fig.add_trace(go.Scatter(
        x=incorrect_hold['Date'],
        y=data.loc[incorrect_hold.index]['Close'],
        mode='markers',
        name='Incorrect Hold Signal',
        marker=dict(symbol='circle', size=6, color='gray', opacity=0.2)
    ))

    # Update layout
    fig.update_layout(
        title=f'{ticker} Stock Price - Actual/Predicted Signals',
        #xaxis_title='Date',
        yaxis_title='Price (USD)',
        template='plotly_dark',
        height=600,
        legend=dict(orientation="h", yanchor="bottom", y=1.02)
    )

    fig.show()

# Call the modified function
plot_combined_predictions(merged_df, 'PLTR')

In [ ]:
from Components.BackTesting import BackTesting
import pandas as pd

#merged_df = pd.read_csv('Data/NASDAQ_100_PredictictionsData.csv')

initial_capital = 10000.0
ticker = 'PLTR'
backtester = BackTesting(merged_df, ticker, initial_capital)
results, _ = backtester.run_simulation()
trades_fig, value_fig, exposure_fig = backtester.plot_performance()
trades_fig.show()
value_fig.show()
exposure_fig.show()

In [ ]:
class TCNBlock(nn.Module):
    def __init__(self, input_dim, output_dim, kernel_size, dilation, padding, dropout=0.2):
        super(TCNBlock, self).__init__()

        self.conv1 = nn.Conv1d(
            in_channels=input_dim,
            out_channels=output_dim,
            kernel_size=kernel_size,
            dilation=dilation,
            padding=padding
        )
        self.norm1 = nn.BatchNorm1d(output_dim)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = nn.Conv1d(
            in_channels=output_dim,
            out_channels=output_dim,
            kernel_size=kernel_size,
            dilation=dilation,
            padding=padding
        )
        self.norm2 = nn.BatchNorm1d(output_dim)
        self.relu2 = nn.ReLU()  # Added missing relu2 activation
        self.dropout2 = nn.Dropout(dropout)

        # Residual connection if dimensions don't match
        self.residual = nn.Conv1d(input_dim, output_dim, 1) if input_dim != output_dim else nn.Identity()

    def forward(self, x):

        # First conv block
        # Residual input
        residual = self.residual(x)

        # First conv block
        out = self.conv1(x)
        out = self.norm1(out)
        out = self.relu1(out)
        out = self.dropout1(out)

        # Second conv block
        out = self.conv2(out)
        out = self.norm2(out)
        out = self.relu2(out)
        out = self.relu2(out)  # Correctly use relu2
        out = self.dropout2(out)

        # Return to original shape
        # Add the residual and pass through final activation
        return self.relu1(out + residual)  # Fixed to use relu1 for the final activation

In [ ]:
class EchoStateNetwork(nn.Module):
    def __init__(self, input_size, reservoir_size, output_size, spectral_radius=0.9,
                 sparsity=0.1, noise=0.001, bidirectional=False):
        super(EchoStateNetwork, self).__init__()

        self.input_size = input_size
        self.reservoir_size = reservoir_size
        self.output_size = output_size
        self.spectral_radius = spectral_radius
        self.sparsity = sparsity
        self.noise = noise
        self.bidirectional = bidirectional

        # Input weights (fixed)
        self.register_buffer('W_in', self._initialize_input_weights())

        # Reservoir weights (fixed)
        self.register_buffer('W', self._initialize_reservoir_weights())

        # Output weights (trainable)
        self.W_out = nn.Linear(reservoir_size, output_size)

        if bidirectional:
            # Second set of weights for backward direction
            self.register_buffer('W_in_reverse', self._initialize_input_weights())
            self.register_buffer('W_reverse', self._initialize_reservoir_weights())
            self.W_out_reverse = nn.Linear(reservoir_size, output_size)
            # Combined output
            self.W_combined = nn.Linear(output_size * 2, output_size)

    def _initialize_input_weights(self):
        W_in = torch.zeros(self.reservoir_size, self.input_size)
        W_in = torch.nn.init.xavier_uniform_(W_in)
        return W_in

    def _initialize_reservoir_weights(self):
        # Create sparse matrix
        W = torch.zeros(self.reservoir_size, self.reservoir_size)
        num_connections = int(self.sparsity * self.reservoir_size * self.reservoir_size)
        indices = torch.randperm(self.reservoir_size * self.reservoir_size)[:num_connections]
        rows = indices // self.reservoir_size
        cols = indices % self.reservoir_size
        values = torch.randn(num_connections)
        W[rows, cols] = values

        # Scale to desired spectral radius
        eigenvalues = torch.linalg.eigvals(W)
        max_eigenvalue = torch.max(torch.abs(eigenvalues))
        W = W * (self.spectral_radius / max_eigenvalue)
        return W

    def _reservoir_step(self, x, h_prev, W_in, W):
        """Execute one step of the reservoir"""
        # h_new = tanh(W_in @ x + W @ h_prev + noise)
        h_new = torch.tanh(torch.mm(x, W_in.t()) + torch.mm(h_prev, W.t()) +
                           self.noise * torch.randn(h_prev.shape, device=h_prev.device))
        return h_new

    def forward(self, x):
        """
        x: input tensor of shape (batch_size, seq_len, input_size)
        """
        batch_size, seq_len, _ = x.size()

        # Forward pass
        h = torch.zeros(batch_size, self.reservoir_size, device=x.device)
        outputs_forward = []

        for t in range(seq_len):
            h = self._reservoir_step(x[:, t], h, self.W_in, self.W)
            outputs_forward.append(self.W_out(h))

        outputs_forward = torch.stack(outputs_forward, dim=1)  # (batch_size, seq_len, output_size)

        if not self.bidirectional:
            return outputs_forward

        # Backward pass for bidirectional ESN
        h_reverse = torch.zeros(batch_size, self.reservoir_size, device=x.device)
        outputs_reverse = []

        for t in range(seq_len - 1, -1, -1):
            h_reverse = self._reservoir_step(x[:, t], h_reverse, self.W_in_reverse, self.W_reverse)
            outputs_reverse.insert(0, self.W_out_reverse(h_reverse))

        outputs_reverse = torch.stack(outputs_reverse, dim=1)  # (batch_size, seq_len, output_size)

        # Combine forward and backward outputs
        combined = torch.cat((outputs_forward, outputs_reverse), dim=2)
        return self.W_combined(combined)

In [ ]:
#ticker.get_balance_sheet(freq='quarterly')
#ticker.get_calendar()
#ticker.get_cash_flow(freq='quarterly')
#earnings_data = ticker.get_earnings_dates()
#income_statement = ticker.get_income_stmt(freq='yearly').T
#ticker.get_institutional_holders()
#ticker.get_recommendations()
#ticker.get_sustainability()

In [ ]:
# define a function to fetch the options data for a given ticker symbol
#def fetch_options_data(ticker_symbol):
    #ticker = yf.Ticker(ticker_symbol)
#    options_dates = ticker.options
#    options_data = ticker.option_chain(date='2025-03-21')
#    return options_data.calls, options_data.puts
##ionq_stock_data = ionq_stock_data.sort_values(by='Date', ascending=False)